In [56]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle

In [57]:
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.svm import SVC

In [58]:
os.chdir("C:\\Users\\Admin\\FingerprintClassification\\model")
from preprocess.preprocessing3 import preprocessing_normal

In [59]:
def read_data(label2num):
    id_anh = 0
    X=[]
    Y=[]
    #output_folder = "C:\\Users\\ADMIN\\FingerprintClassification\\model\\preprocessed_fingerprint"
    for label in os.listdir(os.path.join("dataset\\preprocessed_fingerprint")):
        id_anh+=1
        img = cv2.imread(os.path.join('dataset\\preprocessed_fingerprint',str(id_anh)+'.png'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint"])
        if id_anh>=4000: break
    for label in os.listdir(os.path.join("dataset\\preprocessed_fingerprint")):
        id_anh+=1
        img = cv2.imread(os.path.join('dataset\\preprocessed_fingerprint',str(id_anh)+'.png'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint_noise"])
        if id_anh>=8000: break
    return X,Y


In [60]:
'''def read_data(label2num):
    X = []
    Y = []
    for label in os.listdir(os.path.join("dataset\\fingerprint")):
        img = cv2.imread(os.path.join('dataset\\fingerprint',label))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
        #img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint"])
    for label in os.listdir(os.path.join("dataset\\fingerprint_noise")):
        img = cv2.imread(os.path.join('dataset\\fingerprint_noise',label))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
        #img = img.flatten()
        X.append(img)
        Y.append(label2num["fingerprint_noise"])
    return X,Y'''

'def read_data(label2num):\n    X = []\n    Y = []\n    for label in os.listdir(os.path.join("dataset\\fingerprint")):\n        img = cv2.imread(os.path.join(\'dataset\\fingerprint\',label))\n        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)\n        #img = img.flatten()\n        X.append(img)\n        Y.append(label2num["fingerprint"])\n    for label in os.listdir(os.path.join("dataset\\fingerprint_noise")):\n        img = cv2.imread(os.path.join(\'dataset\\fingerprint_noise\',label))\n        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)\n        #img = img.flatten()\n        X.append(img)\n        Y.append(label2num["fingerprint_noise"])\n    return X,Y'

In [61]:
label2num = {'fingerprint_noise':0, 'fingerprint':1}
X, Y = read_data(label2num)
print(len(X))
print(len(Y))

8000
8000


In [62]:
print(X[0].shape)

(256, 256)


In [63]:
from skimage import feature

In [64]:
cell_size = (16, 16)  # h x w in pixels
block_size = (8, 8)  # h x w in cells
nbins = 4  # number of orientation bins
def hist_vec(X):
    vector_HoG = []
    for i in range(len(X)):
        img = X[i]
        #thresh, img= cv2.threshold(img, 255, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        # 1. Khai báo các tham số
        #cell_size = (8, 8)  # h x w in pixels
        #block_size = (2, 2)  # h x w in cells
        #nbins = 9  # number of orientation bins

        # 2. Tính toán các tham số truyền vào HOGDescriptor
        # winSize: Kích thước của bức ảnh được crop để chia hết cho cell size.
        # blockSize: Kích thước của 1 block
        blockSize = (block_size[1] * cell_size[1], block_size[0] * cell_size[0])
        # blockStride: Số bước di chuyển của block khi thực hiện chuẩn hóa histogram bước 3
        blockStride = (cell_size[1], cell_size[0])

        # 3. Compute HOG descriptor
        hog = cv2.HOGDescriptor(_winSize=img.shape,
                                _blockSize=blockSize,
                                _blockStride=blockStride,
                                _cellSize=cell_size,
                                _nbins=nbins)

        # Kích thước của lưới ô vuông.
        n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])
        
        # Reshape hog feature
        hog_feats = hog.compute(img)\
                    .reshape(n_cells[1] - block_size[1] + 1,
                                n_cells[0] - block_size[0] + 1,
                                block_size[0], block_size[1], nbins) \
                    .transpose((1, 0, 2, 3, 4))  
        #H = feature.hog(img, orientations=9, pixels_per_cell=(8, 8),
        #            cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
        H = hog_feats.flatten()
        vector_HoG.append(H)
        #print('Kích thước hog features: ', H.shape)
    return vector_HoG

In [65]:
vector_HoG = hist_vec(X)

In [66]:
print('Kích thước hog features: ', vector_HoG[0].shape)

Kích thước hog features:  (20736,)


In [67]:
from sklearn.decomposition import PCA
def pca(image_descriptors):
    
    pca = PCA(n_components=400)

    # Fit và transform dữ liệu
    pca.fit(image_descriptors)
    des_pca = pca.transform(image_descriptors)
    return des_pca

In [68]:
pca_HoG = pca(vector_HoG)

In [69]:
print('Kích thước hog features: ', pca_HoG[0].shape)

Kích thước hog features:  (400,)


In [70]:
from sklearn.model_selection import train_test_split, learning_curve, KFold

In [71]:
X_train = []
X_test = []
Y_train = []
Y_test = []
X_valid = []
Y_valid = []
X_train, X_test, Y_train, Y_test = train_test_split(pca_HoG, Y, test_size = 0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.25, random_state=42)

print(len(X_train), " ",len(X_valid), " ", len(X_test))

4800   1600   1600


In [72]:
from sklearn.metrics import accuracy_score

In [73]:
'''# Khởi tạo KFold với k=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Khởi tạo mô hình (ví dụ: Random Forest)
model = LogisticRegression()

# Lưu trữ kết quả
results = []

for train_index, val_index in kf.split(X_train):
    #print(val_index)
    # Chia tập train thành train và validation cho mỗi fold
    #X_train_fold, X_val_fold = X_train[np.array(train_index)], X_train[np.array(val_index)]
    #y_train_fold, y_val_fold = Y_train[np.array(train_index)], Y_train[np.array(val_index)]
    X_train_fold = []
    y_train_fold = []
    X_val_fold = []
    y_val_fold = []
    for sg in train_index:
        X_train_fold.append(X_train[sg])
        y_train_fold.append(Y_train[sg])
    #print(len(X_train_fold), len(X_val_fold), len(y_train_fold), len(y_val_fold))
    for sg in val_index:
        #print(sg)
        X_val_fold.append(X_train[sg])
        y_val_fold.append(Y_train[sg])
    # Huấn luyện mô hình trên tập train_fold
    model.fit(X_train_fold, y_train_fold)

    # Đánh giá trên tập validation
    y_val_pred = model.predict(X_val_fold)
    val_accuracy = accuracy_score(y_val_fold, y_val_pred)
    print(f"Validation accuracy: {val_accuracy}")

    # Đánh giá trên tập test (độc lập)
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(Y_valid, y_test_pred)
    print(f"True Vali accuracy: {test_accuracy}")

    results.append({'val_accuracy': val_accuracy, 'test_accuracy': test_accuracy})

# In kết quả trung bình
print("Average validation accuracy:", sum(result['val_accuracy'] for result in results) / len(results))
print("Average true vali accuracy:", sum(result['test_accuracy'] for result in results) / len(results))'''

'# Khởi tạo KFold với k=5\nkf = KFold(n_splits=5, shuffle=True, random_state=42)\n\n# Khởi tạo mô hình (ví dụ: Random Forest)\nmodel = LogisticRegression()\n\n# Lưu trữ kết quả\nresults = []\n\nfor train_index, val_index in kf.split(X_train):\n    #print(val_index)\n    # Chia tập train thành train và validation cho mỗi fold\n    #X_train_fold, X_val_fold = X_train[np.array(train_index)], X_train[np.array(val_index)]\n    #y_train_fold, y_val_fold = Y_train[np.array(train_index)], Y_train[np.array(val_index)]\n    X_train_fold = []\n    y_train_fold = []\n    X_val_fold = []\n    y_val_fold = []\n    for sg in train_index:\n        X_train_fold.append(X_train[sg])\n        y_train_fold.append(Y_train[sg])\n    #print(len(X_train_fold), len(X_val_fold), len(y_train_fold), len(y_val_fold))\n    for sg in val_index:\n        #print(sg)\n        X_val_fold.append(X_train[sg])\n        y_val_fold.append(Y_train[sg])\n    # Huấn luyện mô hình trên tập train_fold\n    model.fit(X_train_fold, 

In [74]:
lgbm = lgb.LGBMClassifier(class_weight="balanced",num_leaves=15).fit(X_train, Y_train)
print(lgbm.score(X_train, Y_train))
print(lgbm.score(X_valid, Y_valid))
print(lgbm.score(X_test, Y_test))

[LightGBM] [Info] Number of positive: 2395, number of negative: 2405
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010554 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 102000
[LightGBM] [Info] Number of data points in the train set: 4800, number of used features: 400
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
0.9820833333333333
0.805625
0.831875


In [75]:
log = LogisticRegression().fit(X_train, Y_train)
print(log.score(X_train, Y_train))
print(log.score(X_valid, Y_valid))
print(log.score(X_test, Y_test))

0.77875
0.74125
0.76125


In [76]:
sv = SVC(kernel = 'linear')
sv.fit(X_train, Y_train)
print(sv.score(X_train, Y_train))
print(sv.score(X_valid, Y_valid))
print(sv.score(X_test, Y_test))

0.788125
0.728125
0.766875


In [77]:
svp = SVC(kernel = 'poly')
svp.fit(X_train, Y_train)
print(svp.score(X_train, Y_train))
print(svp.score(X_valid, Y_valid))
print(svp.score(X_test, Y_test))

0.8608333333333333
0.69875
0.71875


In [87]:
svr = SVC(kernel = 'rbf', C=11, gamma=0.04)
svr.fit(X_train, Y_train)
print(svr.score(X_train, Y_train))
print(svr.score(X_valid, Y_valid))
print(svr.score(X_test, Y_test))

0.9852083333333334
0.7975
0.783125


In [88]:
from sklearn.metrics import classification_report

y_pred_train = svr.predict(X_train)
y_pred_test = svr.predict(X_test)
# Tính các chỉ số
print("Báo cáo đánh giá trên tập huấn luyện:")
print(classification_report(Y_train, y_pred_train))

print("Báo cáo đánh giá trên tập kiểm tra:")
print(classification_report(Y_test, y_pred_test))

Báo cáo đánh giá trên tập huấn luyện:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2405
           1       0.98      0.99      0.99      2395

    accuracy                           0.99      4800
   macro avg       0.99      0.99      0.99      4800
weighted avg       0.99      0.99      0.99      4800

Báo cáo đánh giá trên tập kiểm tra:
              precision    recall  f1-score   support

           0       0.81      0.74      0.77       796
           1       0.76      0.83      0.79       804

    accuracy                           0.78      1600
   macro avg       0.79      0.78      0.78      1600
weighted avg       0.79      0.78      0.78      1600

